# PDSG ODOT

This notebook explores car crashes in Oregon, from 2003-2016.
This was part of the Portland Data Science Group's Applied Data Science Meetup
for July 2018.

The data was gathered from Oregon Department of Transportation's web portal,
(https://zigzag.odot.state.or.us) using the CDS501 dataset.
This lists nearly every motor vehicle crash in Oregon and includes information about the
time, location, participants, and causes.
In all theres around 700k accidents total.


This is primarily exploratory analysis, with plotting where accidents happen, was alcohol use a factor,
and variation over time.
The goal is to explore the data in Python, perhaps putting some of this data into SQL,
and carry out some Bayesian Statistical Analyses on quantities of interest. 

Does alcohol or drug use increase harm?
Are there seasonal patterns over the day, week, year?

Right now, it also does a simple linear regression on the total number of accidents, with each month handled separately.


## Potential Questions

I'd like to approach this problem partially as a cyclist,
but also as if I were looking into results for the state.

On a personal level I want to know where are roads most dangerous?
At a higher level, I'd like to know the broad trends in time and space,
look at the causes for accidents,
and see if public policy is working (e.g. discouraging drink driving).

### Trend Questions
- How has the number of crashes changed over time?
- How has the number of fatalities changed over time?
(Confounding effects: immigration into the state/cities - need population data)

## Alcohol/Drug questions
- Are incidents with alcohol increasing/decreasing?
- What effect does Alcohol have? (more fatalities? more serious accidents?)
- Seatbelts effect on injuries vs fatalities.

## Location Questions
- Where are the most dangerous places for (drivers, pedestrians, cyclists)?
- Where do the accidents occur (towns, road type, roadways)?

## Externalities
- How does weather affect crashes? (Rural vs Urban?)
- How do accident rates vary as a function of time of day?
- (Get sunset/sunrise information)

What are the most common causes for accidents?


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

RIght now this just looks at a limited set of data relating to each crash.
It would probably make more sense to split this data into two tables, a crash info table,
and a participant info table. 

In [4]:
#Keep columns for location, timing, injuries, alcohol/drugs
#Also demographics (age/sex)
keep=[0,8,9,10,11,12,15,25,26,27,28,29,30,39,40,50,52,53,54,55,67,68,69,
80,81,85,86,87,88,89,90,91,92,93,99,100,127,128]
keep_names=[]
# for i,col in enumerate(df_tot.columns):
#     if (i in keep):
#         keep_names.append(col)
#         print(i,col, 'KEEP')
#     else:
#         print(i,col)

0 Crash ID KEEP
1 Record Type
2 Vehicle ID
3 Participant ID
4 Participant Display Seq#
5 Vehicle Coded Seq#
6 Participant Vehicle Seq#
7 Serial #
8 Crash Month KEEP
9 Crash Day KEEP
10 Crash Year KEEP
11 Week Day Code KEEP
12 Crash Hour KEEP
13 County Code
14 City Section ID
15 Urban Area Code KEEP
16 Functional Class Code
17 NHS Flag
18 Highway Number
19 Highway Suffix
20 Roadway Number
21 Highway Component
22 Mileage Type
23 Connection Number
24 Linear Reference System (LRS)
25 Latitude Degrees KEEP
26 Latitude Minutes KEEP
27 Latitude Seconds KEEP
28 Longitude Degrees KEEP
29 Longitude Minutes KEEP
30 Longitude Seconds KEEP
31 Special Jurisdiction
32 Jurisdiction Group
33 Street Number
34 Nearest Intersecting Street Number
35 Intersection Sequence Number
36 Distance from Intersection
37 Direction From Intersection
38 Milepoint
39 Posted Speed Limit KEEP
40 Road Character KEEP
41 Off Roadway Flag
42 Intersection Type
43 Intersection Related Flag
44 Roundabout Flag
45 Driveway Related

In [29]:
# Load up a few years worth of data from 2002-2005
# Note current Typo: 2002 file is 2016 data?
years=np.arange(2003,2016);
file_dir='./train_data'
df_tot2=pd.DataFrame()
#Add 1 to columns (data processing left in index row?)
keep=[0,8,9,10,11,12,15,25,26,27,28,29,30,39,40,50,52,53,54,55,67,68,69,
80,81,85,86,87,88,89,90,91,92,93,99,100,127,128];
keep2 = [x+1 for x in keep];
for year in years:
    file_name=file_dir+'/'+'SW_Crashes_{}_CDS501.csv'.format(year)
    print('Loading {}'.format(year))
    df_tot2=df_tot2.append( pd.read_csv(file_name,usecols=keep2))

Loading 2015


Loading 2014


Loading 2013


/home/jonathan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (41,68,81) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Loading 2012


...

Loading 2006


Loading 2005


Loading 2004


Loading 2003


Loading 2002


In [30]:
df_tot2.shape

(3338853, 38)

# Checking counts

Count total number of crashes, and number of crashes by number of cyclists involved.

In [31]:
# Unique number of crash IDs
df_tot2['Crash ID'].unique().shape

(669584,)

In [35]:
plt.figure()
plt.subplot(121)
df_tot2['Total Pedalcyclist Count'].hist(log=True,bins=[0,1,2,3,4])
plt.subplot(122)
df_tot2['Total Non-Fatal Injury Count'].hist(log=True,bins=np.arange(15))
plt.show()

So if we assume that cyclists are independent, there's roughly a 100x factor drop between accidents with no cyclists,
and accidents with one and two cyclists.  That suggests 100x as many cars on the road as cyclists.
The comparison against all injuries is primarily visual to see if there's anything anomalous about the cyclists.

Let's look a bit at injuries and fatalities with and without alcohol.

In [63]:
alc_msk=df_tot2['Alcohol-Involved Flag']==1
plt.figure(figsize=(10,6))
plt.subplot(221)
df_tot2.loc[alc_msk,'Total Non-Fatal Injury Count'].hist(log=True,bins=np.arange(12))
plt.subplot(222)
df_tot2.loc[~alc_msk,'Total Non-Fatal Injury Count'].hist(log=True,bins=np.arange(10))
plt.subplot(223)
df_tot2.loc[alc_msk,'Total Fatality Count'].hist(log=True,bins=np.arange(12))
plt.subplot(224)
df_tot2.loc[~alc_msk,'Total Fatality Count'].hist(log=True,bins=np.arange(10))

plt.show()

In [20]:
df_tot2['Sex'].hist()
plt.show()

I believe this is coded as Male=1, Female=2, Unknown (child under 4) =9.

# Crash heatmaps

So let's visualize where the crashes happen.  Nothing fancy.
Just build up a heatmap based on lat/lon pairs for each crash.  Treats each crash as one value.

In [21]:
from heatmap import make_latlon, make_xydict, make_eff_heatmap, make_heatmap_plot

In [36]:
t0=time.time()
lats,lons=make_latlon(df_tot2)
xd=make_xydict(lats,lons,Nx=2000)
heat = make_eff_heatmap(lats,lons,xd,sigma_fac=1)
t1=time.time()
print('time taken',t1-t0)
make_heatmap_plot(heat,xd,'Log10-number of ALL accidents from 2002-2014 in Oregon','crash_heatmap.pdf')

time taken 24.443438053131104


In [37]:
cycle_msk=df_tot2['Total Pedalcyclist Count']>0
df_cycle=df_tot2[cycle_msk]

In [38]:
clats, clons = make_latlon(df_cycle)
cheat=make_eff_heatmap(clats,clons,xd,sigma_fac=1)
make_heatmap_plot(cheat,xd,'Log10 number of BICYCLE accidents from 2012-2015 in Oregon')

So this basically shows that accidents with cyclists are most likely to occur in urban centers.  Which is not really surprising.

There also notable hotpots in the downtown regions in Portland and Eugene.
A rongh estimat for the collision rate would be scale as the square of the density of traffic $n(x)^2$.
One power for having cars at all, and a second power for a collision between vehicles. 


# Group by Year/Month

Let's look at variation by time/year.  And since I got curious, let's look at total crashes and alcohol involved.

In [66]:
df_yr=df_tot2.groupby(['Crash Year','Crash Month'])
df_alc_yr=df_tot2.loc[alc_msk].groupby(['Crash Year','Crash Month'])

In [67]:
#get the count in each year/month pair.
yr_month=df_yr.apply(len)
alc_yr_month=df_alc_yr.apply(len)

In [76]:
plt.figure(figsize=(10,6))
yr_month.plot(label='Total Crashes')
plt.xlabel('(Year, Month)')
plt.ylabel('Total crashes')
plt.show()

There's a clear annual pattern, with a spike around the December/January holidays, when people are likely to be travelling.
2003 seems to be strangely high overall.  I wonder if there was a change in reporting requirements?
Otherwise, there seems to be a growth, probably related to population growth primarily in Portland.
It would be nice to get population data by city across the state over this time period. 

Further questions:
- Split up by city?
- Look at weekends bracketing Chrismas/New Years day?

Some significant changes in driving have occured over this time period: namely the advent of smartphones.
These have perhaps worsened some problems making distracted driving even worse.
This should be accounted for under causes.

What of alcohol?  The Christmas holidays have a lot of parties, typically involving drinking.
Does this show similar variation over the year

In [75]:
plt.figure(figsize=(10,6))
plt.subplot(211)
yr_month.plot(label='Total Crashes')
plt.xlabel('(Year, Month)')
plt.ylabel('Total crashes')
plt.subplot(212)
alc_yr_month.plot(color='orange',label='Total Crashes with Alcohol')
plt.xlabel('(Year, Month)')
plt.ylabel('Total crashes')
plt.show()

The seasonal pattern in crashes with alcohol is much weaker than for crashes as a whole.
But the number of crashes with alcohol seems to have grown faster than crashes over all.
Crashes with alcohol roughly doubled over this 13 year span from 100 to around 200,
while the number of crashes overall grew by 30% or so from 3500 to 5000.

There could conceivably be a shift in around 2010 with what could be a jump from 150 alcohol related collisions to around 200.
I'm more inclined to suspect a change in data collection or some external process rather than a sudden change in drink

In [78]:
yr_month_val=yr_month.values

In [98]:
n=10
#split summer/winter months
months=[0,'Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

summer=[3,4,5,6,7,8]
winter=[1,2,9,10,11,12]
plt.figure()
plt.subplot(211)
for i in summer:
    plt.plot(yr_month_val[i:-1:12],label=months[i])
plt.legend()
tickloc=np.array([0,3,6,9,12])
plt.xticks(tickloc,tickloc+2003)
plt.axis([-1,14,2500,6000])
plt.subplot(212)
for i in winter:
    plt.plot(yr_month_val[i:-1:12],label=months[i])
plt.xlabel('Year')
plt.ylabel('Crashes per month')
plt.axis([-1,14,2500,6000])
tickloc=np.array([0,3,6,9,12])
plt.xticks(tickloc,tickloc+2003)
plt.legend()
plt.show()

In [89]:
np.arange(0,13,3)

array([ 0,  3,  6,  9, 12])

In [87]:
?plt.xticks

So the winter months show the greatest variation between each other and over years.
In comparison the summer months are all fairly closely clustered together, and show the same trend.

Weather seems like the most likely culprit here.
If there's bad weather accidents are more likely since the margins for error are smaller.



# Simple Time Series

Have the number of crashes increased? Well yeah, obviously.

Approaches:
Linear regression month by month.  This pulls out the linear trend.
Treating things month by month removes the main annual seasonality.
This might be a good candidate for a simple SARIMAX model, since we anticipate that crashes overall are increasing too. 

This might also be a good candidate for doing some Bayesian statistics on the regression. 


In [15]:
from sklearn.linear_model import LinearRegression

In [24]:
Nyear=int(len(yr_month)/12)
yr_mval=yr_month.values.reshape(Nyear,12)
#add one to end points coz Python counting
years=np.arange(2003,2016)
years2=np.arange(2003,2017)

In [25]:
years2

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016])

In [21]:
plt.plot(yr_mval.T)
plt.show()

In [72]:
lmod_list=[]
#Run over months, with independent regression for each month
#Going to ditch 2003 as a weird outlier
lo=2004
hi=2015
#only train from 2004-2014.
years =np.arange(2004, hi+1).reshape(-1,1)
#Predict from 2003-2016.
years2=np.arange(2003, hi+2).reshape(-1,1)
#ging to predict one year ahead.
Nyears=len(years2)
pred_tot=np.zeros((Nyears,12))

for i in range(12):
    lmod=LinearRegression()
    lmod.fit(years.reshape(-1,1),yr_mval[1:,i])
    pred=lmod.predict(years2.reshape(-1,1))
    pred_tot[:,i]=pred
    lmod_list.append(lmod)

In [ ]:
#Now train/predict on 2004-2015
for i in range(12):
    lmod=LinearRegression()
    lmod.fit(years.reshape(-1,1),yr_mval[1:-1,i])
    pred=lmod.predict(years2.reshape(-1,1))
    pred_tot[:,i]=pred
    lmod_list.append(lmod)

In [82]:
array_tot=np.zeros((12,4));
array_tot[:,0]=np.arange(12)  #Months
array_tot[:,1]=yr_mval[-1]    #Actual 2015
array_tot[:,2]=pred_tot[-2]   #"pred" 2015
array_tot[:,3]=pred_tot[-1]   #regress 2016

In [83]:
df_pred15=pd.DataFrame(array_tot,columns=['Month','2015-Actual','2015-Regress','2016-Pred'])

In [84]:
df_pred15

    Month  2015-Actual  2015-Regress    2016-Pred
0     0.0       4114.0   4076.923077  4114.409091
1     1.0       3650.0   3723.012821  3792.666667
2     2.0       4009.0   3859.500000  3922.272727
3     3.0       4289.0   3939.807692  4007.000000
4     4.0       4166.0   4133.179487  4210.742424
5     5.0       4898.0   4287.653846  4390.409091
6     6.0       4654.0   4429.679487  4523.787879
7     7.0       4530.0   4320.653846  4383.590909
8     8.0       4701.0   4438.089744  4539.303030
9     9.0       5052.0   4783.346154  4884.818182
10   10.0       5292.0   4807.474359  4904.348485
11   11.0       5801.0   5262.820513  5402.757576

In [86]:
df_pred15.to_csv('Tot_OR_crash.csv',index=False)

In [70]:
for i in range(12):
    plt.plot(np.arange(2003,2016),yr_mval[:,i])
    plt.plot(np.arange(2003,2017),pred_tot[:,i])
gplt.show()

In [88]:
plt.plot(yr_month_val)
plt.plot(pred_tot.reshape(-1))
plt.show()

In [ ]:
# Simple Conditional Probabilities

Let's look at the conditional probabilities for a fatal crash given alcohol, drugs.


## Bayesian Statistics Time

I'd like to apply Bayesian statistics to this problem.
Theres a couple places it could work. First in any regression procedures.
Second in computing the probabilities/likelihoods/risk-factors about the number of accidents involving alcohol, or other factors over time.

If there were some additional data it might be possible to estimate the rate of drink-driving.  